In [ ]:
#Import all the libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from keras.models import Model
from sklearn.metrics import classification_report
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.metrics import confusion_matrix, f1_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
project = pd.read_csv ('Dataset Here')

In [ ]:
project.head()

Comments  Sentiments
0  simply copy and paste the following text into ...           0
1  in order to help increase the booklets downloa...           0
2  as of march 13th 2014 the booklet had been dow...           0
3  click below for a free download of a colorfull...           1
4    click on the download 7 42 mb green banner link           0

In [ ]:
!pip install math
!pip install datatest

In [ ]:
from datatest import validate
from math import isnan

In [ ]:
def removing_features(features, data):
  data.drop(features, inplace = True, axis = 1)

In [ ]:
project.dropna()

In [ ]:
# text cleaning and pre-processing:
def delete_urdu_english_symbols(project):
    cleaned = []
    for sentence in project:
        text = re.sub(r"\d+", " ", sentence)
        text = re.sub(r'\#\.', '', text)
        text = re.sub(r'\n', '', text)
        text = re.sub(r',', '',text)
        text = re.sub(r'\-', ' ',text)
        text = re.sub(r'\.', '',text)
        text = re.sub(r'\\', ' ',text)
        text = re.sub(r'\\x\.+', '',text)
        text = re.sub(r'\d', '',text)
        text = re.sub(r'^_.', '',text)
        text = re.sub(r'_', ' ',text)
        text = re.sub(r'^ ', '',text)
        text = re.sub(r' $', '',text)
        text = re.sub(r'\?', '',text)
        text = re.sub(r'é', '',text)
        text = re.sub(r'§', '',text)
        text = re.sub(r'¦', '',text)
        text = re.sub(r'æ', '',text)
        text = re.sub(r'\d+', '',text)
        text = re.sub('(.*?)\d+(.*?)', '',text)

        # English punctuations
        text = re.sub(r"""[!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]+""", " ", text)
        # Urdu punctuations
        text = re.sub(r"[:؛؟’‘٭ء،۔]+", " ", text)
        # Arabic numbers
        text = re.sub(r"[٠‎١‎٢‎٣‎٤‎٥‎٦‎٧‎٨‎٩]+", " ", text)
        text = re.sub(r"[^\w\s]", " ", text)
        # Remove English characters and numbers.
        text = re.sub(r"[a-zA-z0-9]+", " ", text)

        text = re.sub(r" +", " ", text)
        text = text.split(" ")
        # some stupid empty tokens should be removed.
        text = [t.strip() for t in text if t.strip()]
        cleaned.append(" ".join(text))
    return cleaned


In [ ]:

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)

    return emoji_pattern.sub(r'', text)




In [ ]:
#preprocess train dataset
project['Comments'] = project['Comments'].apply(remove_emoji)



In [ ]:
# split the dataset into training and testing datasets
x_train, x_test, y_train, y_test = train_test_split(project['Comments'], project['Sentiments'], test_size=0.1, random_state=1)#
# create a count vectorizer object for tokenization and preprocessing
cv = CountVectorizer(stop_words='english')#create an object to fit
cv.fit(project['Comments'])
# transform the training and testing data using count vectorizer object(test counts,tfidf and ngrams)
x_train_plus_count =  cv.transform(x_train)#with simple count vectors
x_test_plus_count =  cv.transform(x_test)
#for further using model +countvectors + tf-idf
tfidf_tr = TfidfTransformer()
x_train_plus_count_plus_tfidf = tfidf_tr.fit_transform(x_train_plus_count)
x_test_plus_count_plus_tfidf = tfidf_tr.transform(x_test_plus_count)

# ngram level tf-idf (model plus ngram)
tv_ngram = TfidfVectorizer(ngram_range=(1,2))
tv_ngram.fit(project['Comments'])
x_train_tfidf_ngram =  tv_ngram.transform(x_train)
x_test_tfidf_ngram =  tv_ngram.transform(x_test)

In [ ]:
#A class for our videomodel
def videomodel(classifier, Xtrain, Ytrain, Xtest):
    classifier.fit(Xtrain, Ytrain)
    predictions = classifier.predict(Xtest)
    print("\nAccuracy : ", accuracy_score(predictions, y_test))

    target_names = ['class 0', 'class 1']
    print(classification_report(y_test, predictions, target_names=target_names))

In [ ]:
print("\nCount Vectors used with simple Naive Bayes ")
videomodel(MultinomialNB(), x_train_plus_count, y_train, x_test_plus_count)

print("\nCount vectors and Term frequency-inverse document frequency(tf-idf) with multinomial naive bayes ")
videomodel(MultinomialNB(), x_train_plus_count_plus_tfidf, y_train, x_test_plus_count_plus_tfidf)

print("\nN-Gram Vectors alongwith Naive Bayes ")
videomodel(MultinomialNB(), x_train_tfidf_ngram, y_train, x_test_tfidf_ngram)

In [ ]:
print("\nRandom Forrest Classifier with Count Vectors: ")
videomodel(RandomForestClassifier(random_state=40,n_estimators=2000), x_train_plus_count, y_train, x_test_plus_count)

print("\nRandom Forrest Classifier with Count Vectors and using TF-IDF: ")
videomodel(RandomForestClassifier(random_state=40,n_estimators=2000), x_train_plus_count_plus_tfidf, y_train, x_test_plus_count_plus_tfidf)

print("\nRandom Forrest Classifier and just N-Gram Vectors: ")
videomodel(RandomForestClassifier(random_state=40,n_estimators=2000), x_train_tfidf_ngram, y_train, x_test_tfidf_ngram)

In [ ]:
print("\nLinearSVM and Count Vectors : ")
videomodel(SVC(kernel='linear'), x_train_plus_count, y_train, x_test_plus_count)

print("\nLinearSVM CountVectors and Term Frequency Inverse Document Frequency ")
videomodel(SVC(kernel='linear'), x_train_plus_count_plus_tfidf, y_train, x_test_plus_count_plus_tfidf)

print("\nLinearSVM with the N-Gram Vector ")
videomodel(SVC(kernel='linear'), x_train_tfidf_ngram, y_train, x_test_tfidf_ngram)

In [ ]:
import sklearn.metrics as metrics

NB with CountVectors

In [ ]:
naivebayes = MultinomialNB()
naivebayes.fit(x_train_plus_count, y_train)
predictions = naivebayes.predict(x_test_plus_count)
print("\nTheAccuracy : ", accuracy_score(predictions, y_test))

probs = naivebayes.predict_proba(x_test_plus_count)
preds = probs[:,1]
fpr_nb, tpr_nb, threshold_nb = metrics.roc_curve(y_test, preds)
roc = metrics.auc(fpr_nb, tpr_nb)

import matplotlib.pyplot as plt
plt.title('MultinomialNaive bayes Roc Curve 1')
plt.plot(fpr_nb, tpr_nb, 'b', label = 'AUC = %0.2f' % roc)
plt.legend(loc = 'center left')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive')
plt.xlabel('False Positive')
plt.show()

SVM with Count Vectors

In [ ]:
svm = SVC(kernel = 'linear', probability=True)
svm.fit(x_train_plus_count, y_train)
predictions = svm.predict(x_test_plus_count)
print("\nTheAccuracy : ", accuracy_score(predictions, y_test))

probs = svm.predict_proba(x_test_plus_count)
preds = probs[:,1]
fpr_svm, tpr_svm, threshold_svm = metrics.roc_curve(y_test, preds)
roc = metrics.auc(fpr_svm, tpr_svm)

import matplotlib.pyplot as plt
plt.title('Linear SVM Roc Curve 1')
plt.plot(fpr_svm, tpr_svm, 'b', label = 'AUC = %0.2f' % roc)
plt.legend(loc = 'center left')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive')
plt.xlabel('False Positive')
plt.show()

RFC with CountVectors

In [ ]:
randf = RandomForestClassifier(random_state=40,n_estimators=2000)
randf.fit(x_train_plus_count, y_train)
predictions = randf.predict(x_test_plus_count)
print("\nTheAccuracy : ", accuracy_score(predictions, y_test))

probs = randf.predict_proba(x_test_plus_count)
preds = probs[:,1]
fpr_rf, tpr_rf, threshold_rf = metrics.roc_curve(y_test, preds)
roc = metrics.auc(fpr_rf, tpr_rf)

import matplotlib.pyplot as plt
plt.title('Random Forrest Classifier Roc Curve 1')
plt.plot(fpr_rf, tpr_rf, 'b', label = 'AUC = %0.2f' % roc)
plt.legend(loc = 'center left')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive')
plt.xlabel('False Positive')
plt.show()

NB with TFIDF

In [ ]:
naivebayes = MultinomialNB()
naivebayes.fit(x_train_plus_count_plus_tfidf, y_train)
predictions = naivebayes.predict(x_test_plus_count_plus_tfidf)
print("\nThe Accuracy  ", accuracy_score(predictions, y_test))

probs = naivebayes.predict_proba(x_test_plus_count_plus_tfidf)
preds = probs[:,1]
fpr_nb, tpr_nb, threshold_nb = metrics.roc_curve(y_test, preds)
roc = metrics.auc(fpr_nb, tpr_nb)

import matplotlib.pyplot as plt
plt.title('Multinomial naive Bayes Roc Curve 2')
plt.plot(fpr_nb, tpr_nb, 'b', label = 'AUC = %0.2f' % roc)
plt.legend(loc = 'center left')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive')
plt.xlabel('False Positive')
plt.show()

SVM with TFIDF

In [ ]:
svm = SVC(kernel = 'linear', probability=True)
svm.fit(x_train_plus_count_plus_tfidf, y_train)
predictions = svm.predict(x_test_plus_count_plus_tfidf)
print("\nTheAccuracy : ", accuracy_score(predictions, y_test))

probs = svm.predict_proba(x_test_plus_count_plus_tfidf)
preds = probs[:,1]
fpr_svm, tpr_svm, threshold_svm = metrics.roc_curve(y_test, preds)
roc = metrics.auc(fpr_svm, tpr_svm)

import matplotlib.pyplot as plt
plt.title('Linear SVM Roc Curve 2')
plt.plot(fpr_svm, tpr_svm, 'b', label = 'AUC = %0.2f' % roc)
plt.legend(loc = 'center left')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive')
plt.xlabel('False Positive')
plt.show()

RFC with TF IDF

In [ ]:

randf = RandomForestClassifier(random_state=40,n_estimators=2000)
randf.fit(x_train_plus_count_plus_tfidf, y_train)
predictions = randf.predict(x_test_plus_count_plus_tfidf)
print("\nTheAccuracy : ", accuracy_score(predictions, y_test))

probs = randf.predict_proba(x_test_plus_count_plus_tfidf)
preds = probs[:,1]
fpr_rf, tpr_rf, threshold_rf = metrics.roc_curve(y_test, preds)
roc = metrics.auc(fpr_rf, tpr_rf)

import matplotlib.pyplot as plt
plt.title('Random Forrest Classifier Roc Curve 2')
plt.plot(fpr_rf, tpr_rf, 'b', label = 'AUC = %0.2f' % roc)
plt.legend(loc = 'center left')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive')
plt.xlabel('False Positive')
plt.show()

NB with n gram

In [ ]:
naivebayes = MultinomialNB()
naivebayes.fit(x_train_tfidf_ngram, y_train)
predictions = naivebayes.predict(x_test_tfidf_ngram)
print("\nThe Accuracy  ", accuracy_score(predictions, y_test))

probs = naivebayes.predict_proba(x_test_tfidf_ngram)
preds = probs[:,1]
fpr_nb, tpr_nb, threshold_nb = metrics.roc_curve(y_test, preds)
roc = metrics.auc(fpr_nb, tpr_nb)

import matplotlib.pyplot as plt
plt.title('Multinomial naive Bayes Roc Curve 3')
plt.plot(fpr_nb, tpr_nb, 'b', label = 'AUC = %0.2f' % roc)
plt.legend(loc = 'center left')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive')
plt.xlabel('False Positive')
plt.show()

SVM with N gram

In [ ]:
svm = SVC(kernel = 'linear', probability=True)
svm.fit(x_train_tfidf_ngram, y_train)
predictions = svm.predict(x_train_tfidf_ngram)
print("\nTheAccuracy : ", accuracy_score(predictions, y_test))

probs = svm.predict_proba(x_test_tfidf_ngram)
preds = probs[:,1]
fpr_svm, tpr_svm, threshold_svm = metrics.roc_curve(y_test, preds)
roc = metrics.auc(fpr_svm, tpr_svm)

import matplotlib.pyplot as plt
plt.title('Linear SVM Roc Curve 3')
plt.plot(fpr_svm, tpr_svm, 'b', label = 'AUC = %0.2f' % roc)
plt.legend(loc = 'center left')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive')
plt.xlabel('False Positive')
plt.show()

RFC with N gram

In [ ]:
randf = RandomForestClassifier(random_state=40,n_estimators=2000)
randf.fit(x_train_tfidf_ngram, y_train)
predictions = randf.predict(x_test_tfidf_ngram)
print("\nTheAccuracy : ", accuracy_score(predictions, y_test))

probs = randf.predict_proba(x_test_tfidf_ngram)
preds = probs[:,1]
fpr_rf, tpr_rf, threshold_rf = metrics.roc_curve(y_test, preds)
roc = metrics.auc(fpr_rf, tpr_rf)

import matplotlib.pyplot as plt
plt.title('Random Forrest Classifier Roc Curve 3')
plt.plot(fpr_rf, tpr_rf, 'b', label = 'AUC = %0.2f' % roc)
plt.legend(loc = 'center left')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive')
plt.xlabel('False Positive')
plt.show()